In [62]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

from src.features import build_features

# Define dictionaries for column names and types
PROCEDURE_COLUMNS = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
SERVICE_COLUMNS = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}
DATE_COLUMNS = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}

CAMBIO_CAMAS = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}

# Define relevant diagnoses
RELEVANT_DIAGNOSES = [
    "I35.0",
    "C34.1",
    "C34.2",
    "C34.3",
    "C78.0",
    "C78.2",
    "D38.1",
    "I71.2",
    "I71.0",
    "I35.1",
    "Q20.1",
    "Q20.2",
    "Q20.3",
    "Q20.4",
    "Q20.5",
    "Q20.6",
    "Q20.8",
    "Q20.9",
    "Q21.0",
    "Q21.1",
    "Q21.2",
    "Q21.3",
    "Q21.4",
    "Q21.8",
    "Q22.0",
    "Q22.1",
    "Q22.2",
    "Q22.3",
    "Q22.4",
    "Q22.5",
    "Q22.8",
    "Q23.0",
    "Q23.1",
    "Q23.3",
    "Q24.0",
    "Q24.1",
    "Q24.4",
    "Q24.5",
    "Q24.6",
    "Q24.8",
    "Q24.9",
    "Q25.0",
    "Q25.1",
    "Q25.3",
    "Q25.4",
    "Q25.5",
    "Q25.6",
    "Q25.7",
    "Q25.8",
    "Q25.9",
    "Q26.4",
    "Q26.8",
    "Q27.2",
    "Q27.3",
    "Q28.8",
    "Q28.9",
    "Q31.1",
    "Q32.0",
    "Q32.1",
    "Q33.0",
    "Q33.1",
    "Q33.2",
    "Q33.4",
    "Q33.8",
    "Q34.1",
    "Q34.8",
    "Q67.8",
    "Q76.5",
    "Q76.6",
    "Q76.7",
    "Q76.8",
    "Q76.9",
    "Q78.0",
    "Q79.0",
    "Q79.8",
    "Q83.9",
    "Q85.0",
    "Q85.8",
    "Q85.9",
    "Q87.4",
    "Q89.3",
    "I36.1",
]

In [64]:
# Lee la base de datos GRD del INT
df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")
df_torax = df_torax.reset_index(drop=True).reset_index(names="id_egreso")

C:\Users\javie\AppData\Local\Temp\ipykernel_26192\3157398680.py:2: DtypeWarning: Columns (85,86,88,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")


In [65]:
# Convierte los traslados y sus fechas a formato long
viaje_paciente = build_features.crear_viaje_paciente(df_torax, SERVICE_COLUMNS, DATE_COLUMNS)

# Procesa el viaje del paciente, ordenando las fechas de traslado y eliminando las que no se imputan
viaje_paciente = build_features.procesar_viaje_paciente(
    viaje_paciente, SERVICE_COLUMNS, ["id_egreso", "tipo_servicio"], CAMBIO_CAMAS
)

# Calcula los dias de estadia por cada uno de los servicios
viaje_paciente = build_features.procesar_duracion_estadia(viaje_paciente)

In [75]:
# Obtiene el resumen de dias de estada totales por cada uno de los diags
resumen_dias_de_estada = build_features.procesar_resumen_dias_estada(viaje_paciente)

In [69]:
resumen_dias_de_estada_wide_relevantes = resumen_dias_de_estada_wide[
    resumen_dias_de_estada_wide[("DIAGNOSTICO1", "")].isin(RELEVANT_DIAGNOSES)
]

In [70]:
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico_DE.xlsx") as file:
    resumen_dias_de_estada.to_excel(file, sheet_name="long")
    resumen_dias_de_estada_wide.to_excel(file, sheet_name="wide")
    resumen_dias_de_estada_wide_relevantes.to_excel(file, sheet_name="wide_relevantes")